<a href="https://colab.research.google.com/github/burakbasher/nextjs-xray-website/blob/main/Model_E%C4%9Fitim_Kodu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GPU Durumu Kontrolü ve Dizin Ayarları**

Bu bölümde, sistemde mevcut bir GPU'nun olup olmadığı ve CUDA desteğinin durumu kontrol edilmektedir. Eğer varsa, model eğitim ve tahmin işlemleri GPU kullanılarak hızlandırılabilir. Ayrıca, çalışma dizini de alınarak ilerleyen adımlarda kullanılmak üzere HOME değişkenine atanmıştır.



In [ ]:
# GPU ve CUDA durumunu kontrol etmek için kullanılır. NVIDIA GPU'nun mevcut olup olmadığını ve kullanım durumu gösterilir.
!nvidia-smi

# Geçerli çalışma dizinini alır ve HOME değişkenine atar.
import os
HOME = os.getcwd()
print(HOME)

Sat Aug 31 18:02:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              47W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

**Yolov9 Klonlanması ve Parametrelerin İçe Aktarılması**

Bu kısımda, YOLOv9 modelinin kodları GitHub'dan klonlanır ve bağımlılıkları yüklenir. Eğitim ve tahmin işlemlerinde kullanılacak model ağırlıkları indirildikten sonra weights dizininde saklanır. İndirilen her bir ağırlık dosyası farklı bir model yapılandırması için kullanılabilir.

In [ ]:
# Yolov9 modelinin GitHub deposundan klonlanması. Bu adımda modelin kodlarına erişim sağlanır.
!git clone https://github.com/SkalskiP/yolov9.git
# Yolov9 dizinine geçiş yapılır.
# %cd yolov9
# Gerekli bağımlılıkları yüklemek için requirements.txt dosyasındaki paketler yüklenir.
!pip install -r requirements.txt -q

# Roboflow kütüphanesi ve görsel gösterimi için gerekli kütüphaneler yüklenir.
import roboflow
from IPython.display import Image

# Ağırlıkları depolamak için 'weights' dizini oluşturulur.
!mkdir -p {HOME}/weights

# Yolov10 ağırlıkları indiriliyor. Bu ağırlıklar eğitim ve tahmin işlemleri için kullanılacak.
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-e.pt

# Ağırlık dosyalarının bulunduğu dizini listeler ve kontrol eder.
!ls -la {HOME}/weights


/content


**Veri Seti İçe Aktarılması**

Bu bölümde, Roboflow aracılığıyla bir veri seti indirilir. Veri seti, YOLOv9 formatında eğitilecek şekilde yapılandırılmıştır. Roboflow API'si, projeye erişim sağlamak için kullanılır ve ilgili X-ray veri seti projeden alınır. Bu veri seti, modelin eğitimi için kullanılacaktır.



In [ ]:
# Datasetler için yeni bir dizin oluşturuluyor.
!mkdir {HOME}/datasets
# %cd {HOME}/datasets

# Roboflow kütüphanesi yükleniyor.
!pip install -q roboflow

# Roboflow API kullanılarak veri seti indiriliyor.
from roboflow import Roboflow
rf = Roboflow(api_key="YOUR API")
project = rf.workspace("xray-c2nkx").project("x-ray-kxi5k")
version = project.version(24)
dataset = version.download("yolov9")


**Model Eğitimi**

Bu kısımda model eğitimi gerçekleştirilir. Eğitim sırasında kullanılacak veri seti, ağırlık dosyaları, yapılandırma dosyası ve hiperparametreler belirlenir. batch, epochs, img, ve device gibi parametrelerle eğitim yapılandırması özelleştirilir. Bu eğitim işlemi sırasında, model X-ray görüntüleri üzerinde eğitilerek nesne tanıma yeteneklerini geliştirir.

In [ ]:
# Çalışma dizinine geçiş yapılır.
# %cd {HOME}

# YOLOv9 dizinine geçiş yapılarak eğitim işlemi başlatılır.
# Aşağıda batch size, epoch sayısı, görüntü boyutu, cihaz (GPU), ve kullanılan ağırlık dosyaları gibi parametreler belirtilmiştir.
!python /content/yolov9/train.py \
    --batch 16 --epochs 100 --img 640 --device 0 --min-items 0 --close-mosaic 15 \
    --data /content/datasets/X-ray-24/data.yaml \
    --weights /content/weights/gelan-c.pt \
    --cfg /content/yolov9/models/detect/gelan-c.yaml \
    --hyp hyp.scratch-high.yaml


**Model Değerlendirmesi**

Bu bölümde, modelin eğitim sürecinde elde edilen değerlendirme sonuçları görselleştirilir. Karmaşıklık matrisi ve sonuç grafiği, modelin doğruluk ve başarımını görsel olarak analiz etmek için kullanılır. Böylece modelin ne kadar iyi performans gösterdiği kolayca incelenebilir.



In [ ]:
# Eğitilen modelin değerlendirme sonuçlarının görselleştirilmesi.
# Confusion matrix (karmaşıklık matrisi) ve sonuç grafiği görüntülenir.
from IPython.display import Image

Image(filename=f'/content/yolov9/runs/train/exp/confusion_matrix.png', width=600)
Image(filename=f'/content/yolov9/runs/train/exp/results.png', width=600)


 **Tahminler**

Bu adımda, test edilmek üzere bir dizinde bulunan tüm .jpg formatındaki resim dosyalarının yolları alınır. Tahmin işlemi bu dosyalar üzerinde gerçekleştirilecektir.



In [ ]:
# Test klasöründeki tüm .jpg dosyalarını listelemek için kullanılır.
import os
from PIL import Image

image_dir = '/content/tests'
image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.endswith('.jpg')]

image_paths


**Model ile Tahmin Yapılması**

Bu kısımda, eğitilmiş model kullanılarak test görüntüleri üzerinde tahminler gerçekleştirilir. Tahmin sonuçları belirlenen klasöre kaydedilir. Tahminlerde kullanılan ağırlık dosyası, eğitim sürecinde elde edilen en iyi performans gösteren modeldir.

In [ ]:
%cd /content/yolov9

In [ ]:
# YOLOv9 dizinine geçiş yapılır ve tahmin işlemi başlatılır.
# Eğitimden elde edilen en iyi ağırlık dosyası kullanılarak tahmin yapılır.
# Görüntüler test klasöründen alınır ve sonuçlar 'results' adında bir klasöre kaydedilir.
# %cd /content/yolov9

!python detect.py \
    --weights /content/yolov9/runs/train/exp/weights/best.pt \
    --source /content/tests\
    --img 640 \
    --conf 0.25 \
    --device 0 \
    --name results


**Sonuçların Sıkıştırılması ve İndirilmesi**

Bu bölümde, tahmin sonuçları ve model ağırlıkları zip formatında sıkıştırılır ve yerel bilgisayara indirilebilir hale getirilir. Bu adım, model çıktılarının kolayca paylaşılmasını veya saklanmasını sağlar.

In [ ]:
# Tahmin sonuçlarının bulunduğu klasör zip formatında sıkıştırılır ve indirilmek üzere hazırlanır.
import shutil

# Tahmin sonuçlarını sıkıştır.
shutil.make_archive('/content/weights', 'zip', '/content/yolov9/runs/train/exp/weights')

from google.colab import files

# Sıkıştırılmış dosyayı indirin.
files.download('/content/weights.zip')


**Model Ağırlıklarının Kaydedilmesi**

Son olarak, eğitilen modelin ağırlıkları kaydedilir. Bu adımda model, daha sonra yeniden kullanılmak üzere ağırlıklarıyla birlikte saklanır.

In [ ]:
import torch

# Model ağırlıklarını bir dosyaya kaydedin.
torch.save(model.state_dict(), 'model_weights_only.pt')
